In [10]:
!pip install simpletransformers

  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=52ebf7562b9523456c50ebc5d75357b1a5f31a329ef525f30461ad7cc2576213
  Stored in directory: c:\users\rober\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=f57cb705c2a388e03023f90b60be39156dfd6ba59912c0307e0fd30f6d6b421e
  Stored in directory: c:\users\rober\appdata\local\pip\cache\wheels\9f\69\d1\50b39b308a87998eaf5c1d9095e5a5bd2ad98501e2b7936d36
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=b39ac3b4d2d02900ae614fe099fa8778be416eeeedd0881a8498264f2ef29262
  Stored in directory: c:\users\rober\appdata\local\pip\cache\wheels\4c\8e\7e\72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=e0dd

In [11]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
# !git clone https://github.com/EnnedueJ/Text-Analytics-Project.git

In [4]:
import pandas as pd

df = pd.read_csv('movie_dataset_classification.csv',index_col=0)
df.dropna(subset=['Plot'], inplace=True)
df.drop_duplicates(subset=['Title','Plot'], inplace=True)
#removing low frequency genres
counts = df.Genre.value_counts()
counts = list(counts[counts < 150].index)
df = df[~df['Genre'].isin(counts)]

In [5]:
df

,Release Year,Title,Director,Cast,Plot,Genre
0,1903.0,The Great Train Robbery,Edwin S. Porter,NaN,The film opens with two bandits breaking into ...,western
1,1904.0,The Suburbanite,Wallace McCutcheon,NaN,The film is about a family who move to the sub...,comedy
2,1906.0,From Leadville to Aspen: A Hold-Up in the Rockies,Francis J. Marion and Wallace McCutcheon,NaN,The film features a train traveling through th...,action
3,1907.0,Daniel Boone,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",Boone's daughter befriends an Indian maiden as...,historical
4,1907.0,How Brown Saw the Baseball Game,Unknown,Unknown,Before heading out to a baseball game at a nea...,comedy
...,...,...,...,...,...,...
40117,NaN,Ready Player One,NaN,[],Wade Watts is a teenager who lives in a slum w...,sport
40118,2011.0,From the Rough,Pierre Bagley,"['Taraji P. Henson', 'Tom Felton', 'Michael Cl...",Catana Starks is the coach for the women's swi...,sport
40119,NaN,Brink!,Greg Beeman,"['Erik von Detten', 'Christina Vidal', 'Robin ...","Andy ""Brink"" Brinker and his in-line skating c...",sport
40120,2003.0,Open Water (film),Chris Kentis,"['Blanchard Ryan', 'Daniel Travis']",Daniel Kintner and Susan Watkins are frustrate...,sport


In [6]:
from sklearn.model_selection import train_test_split

X = df['Plot'].values
y = df['Genre'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, shuffle=True)

In [7]:
(train_inputs, validation_inputs,
 train_labels, validation_labels) = train_test_split(X, y,
                                                     random_state=42,
                                                     test_size=0.1)
train_df = pd.DataFrame({'text':train_inputs,'label':train_labels})
validation_df = pd.DataFrame({'text':validation_inputs,'label':validation_labels})
train_df,validation_df

(                                                    text            label
 0      The Périgord region of southwest France has be...      documentary
 1      The two lead characters appear as the robotic ...  science fiction
 2      A poor boy falls in love with a rich girl but ...          romance
 3      The film opens with a parody of Paramount's 19...        animation
 4      Pushpa and Gayatri are the daughters of an ex-...            drama
 ...                                                  ...              ...
 33637  Esther Blueburger is a 13-year-old Jewish outc...            drama
 33638  Lou Henry (Lou Costello) is the owner of Kiddy...           comedy
 33639  Brothers Bill and Dennis reunite after their a...            drama
 33640  Anita Morgan, a descendant of the famous pirat...            drama
 33641  Imogene Duncan (Kristen Wiig) was once a promi...           comedy
 
 [33642 rows x 2 columns],
                                                    text      label
 0 

In [13]:
# Optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 4
model_args.overwrite_output_dir = True

In [14]:
# Create a ClassificationModel
model = ClassificationModel("bert", "bert-base-uncased",args=model_args)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
model.train_model(train_df,eval_df=validation_df,model_args=model_args)